In [1]:
import os
import pickle
from datetime import datetime, timezone
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd

import matplotlib.colors
import numpy as np

import xarray as xr
import xroms
import matplotlib.pyplot as plt
from wavespectra import read_ww3, read_swan, read_ndbc, read_netcdf
#from roguewave.wavespectra.estimators.estimate import estimate_directional_distribution
#%run -i roguewave_estimators.py
%run -i wave_stats.py

In [2]:
roms_file = 'http://icoast.rc.ufl.edu/thredds/dodsC/ian/L1_BEST_AVAILABLE/ocean_his.nc'
run_name = 'L1_20230123'

ds_mod = xroms.open_netcdf(roms_file)
time_ini_model = ds_mod.ocean_time.values[0]
time_end_model = ds_mod.ocean_time.values[-1]

OSError: [Errno -90] NetCDF: file not found: 'http://icoast.rc.ufl.edu/thredds/dodsC/ian/L1_BEST_AVAILABLE/ocean_his.nc'

In [3]:
# These are the GOM buoys with data for Ian
buoys = ['42039','42098', '42099']
# pick one for testing
buoy = buoys[0]

In [4]:
MODEL_DIRECTORY = '/vortexfs1/share/usgs-share/Projects/Ian2022/ian10/drifters/'

In [5]:
# Process one buoy as a test
fn = "{}.spc2d".format(buoy)
dsm = read_swan(MODEL_DIRECTORY+fn)
# get start/end times
stime = dsm.time[0].values
etime = dsm.time[-1].values
print(stime,' to ',etime)
dtmod = np.squeeze( dsm.time.values )
hsmod = np.squeeze( dsm.efth.spec.hs().values )

In [5]:
# assume the file has all years, thus w9999 instead of w2022
# read the NDBC buoy observations for the same time slice
url ='https://dods.ndbc.noaa.gov/thredds/dodsC/data/swden/' + str(buoy) + '/' + str(buoy) + 'w9999' + '.nc'
print(url)
dso = read_ndbc(url, dd=10.0, chunks={}).sel(time=slice(stime, etime))
dtobs = np.squeeze(dso.time)
hsobs = np.squeeze(dso.efth.spec.hs().values)
dmobs = np.squeeze(dso.efth.spec.dm().values)
dsprobs = np.squeeze(dso.efth.spec.dspr().values)
sweobs = np.squeeze(dso.efth.spec.swe().values)
swobs = np.squeeze(dso.efth.spec.sw().values)
print('dmobs:', dmobs[80])
print('dspr:', dsprobs[80])

# interpolate modeled to times of observation (in case some are missing)
dsmi = dsm.interp(time=dso.time, method='nearest')
hsmodi = np.squeeze( dsmi.efth.spec.hs().values )
rmse = np.sqrt( np.nanmean( (hsobs - hsmodi)**2) )
print(rmse)

2022-09-26T00:00:00.000000000  to  2022-10-02T00:00:00.000000000
https://dods.ndbc.noaa.gov/thredds/dodsC/data/swden/42039/42039w9999.nc


/vortexfs1/home/csherwood/mambaforge/envs/NOPP/lib/python3.9/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


dmobs: nan
dspr: nan
0.2072119026495582


/vortexfs1/home/csherwood/mambaforge/envs/NOPP/lib/python3.9/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


In [6]:
hsobs

array([0.47495262, 0.4622986 , 0.45181854, 0.44710178, 0.41997618,
       0.40144738, 0.43169433, 0.40735734, 0.38118236, 0.3728002 ,
       0.34397674, 0.37140276, 0.36935079, 0.4017462 , 0.37981575,
       0.43848831, 0.43769854, 0.36138624, 0.35449682, 0.3490215 ,
       0.37841776, 0.38675057, 0.38835037, 0.41208252, 0.38108791,
       0.42222269, 0.40792158, 0.3489699 , 0.36937787, 0.39605557,
       0.43924935, 0.60468176, 0.90551641, 1.1880404 , 1.24370413,
       1.34473786, 1.43902051, 1.54583309, 1.54338584, 1.5983554 ,
       1.44305228, 1.55128335, 1.4061579 , 1.43924981, 1.42375563,
       1.50630674,        nan,        nan, 1.76055673,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        

In [ ]:
dso.isel(time=80).spec.hs().values

In [ ]:
url ='https://dods.ndbc.noaa.gov/thredds/dodsC/data/swden/' + str(buoy) + '/' + str(buoy) + 'w9999' + '.nc'
ds = xr.open_dataset(url)
time_ini_model = datetime.strptime('2022-09-26', '%Y-%m-%d')
time_end_model = datetime.strptime('2022-10-01', '%Y-%m-%d')
ds = ds.sel(time=slice(time_ini_model,time_end_model))
ds

In [ ]:
#for i in np.arange(len(ds.time)):
i=80
f = np.squeeze( ds.frequency )
a1, b1, a2, b2 = to_moment( ds.wave_spectrum_r1[i], ds.wave_spectrum_r1[i], ds.mean_wave_dir[i], ds.principal_wave_dir[i] )
spec1d = np.squeeze( ds.spectral_wave_density[i] )
thetamw = calc_weighted_theta_mean_a1b1( a1, b1, spec1d, f )
sigmatheta = calc_sigma_theta_a1b1( a1, b1, spec1d, f )
print(thetamw, sigmatheta)

In [ ]:
plt.plot(f,np.squeeze(spec1d))
plt.plot(f,np.squeeze(a1))
plt.plot(f,np.squeeze(b1))

In [ ]:
time_ini_model = datetime.strptime('2022-09-26', '%Y-%m-%d')
time_end_model = datetime.strptime('2022-10-01', '%Y-%m-%d')
buoys = "42097","42099","42098"

for buoy in buoys:
     # Open dataset
     url ='https://dods.ndbc.noaa.gov/thredds/dodsC/data/stdmet/' + str(buoy) + '/' + str(buoy) + 'h9999.nc'
     ds = xr.open_dataset(url)
     lat1 = ds.latitude.values
     lon1 = ds.longitude.values
     print(lon1,lat1)
     ds = ds.sel(time=slice(time_ini_model,time_end_model)) ####<<<<<We can us the datetime object to subset. no need to convert to strings. 

     #HWavel = ds_mod.Hwave.xroms.sel2d(lon1, lat1)

     plt.figure()
     ds.wave_height.plot()
     #HWavel.plot()
     plt.title('NDBC buoy observations')
     plt.ylabel('Significat Wave height, m')
     plt.xlabel('YYYY-MM-DD, UTC')
     #plt.gca().legend([buoy, "L1 COAWST"])
     fig_name = buoy + '_v2_' + run_name + '.png'
     #plt.savefig(fig_name)

In [ ]:
ds